In [ ]:
from boardlaw.sql import *

In [ ]:
from analysis import data
ags = data.load()

In [ ]:
import plotnine as pn
from analysis import plot

noise = query('''select * from noise_scales''')
df = pd.merge(noise, ags, left_on='id', right_on='snap_id').query('test_nodes == 64')

(pn.ggplot(df)
     + pn.geom_point(pn.aes(x='data.ELO*elo', y='np.log10(S/G2)', color='factor(boardsize)'), shape='.', size=.25, show_legend=False)
     + pn.facet_wrap('boardsize', ncol=2)
    + plot.IEEE()
    + pn.theme(figure_size=(4, 4)))

In [ ]:
df